In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import os
from datetime import datetime
from typing import Any, List
import pandas as pd
import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes
from attrs import define, field, Factory

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList
from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList
from phoglobushelpers.compatibility_objects.Tasks import FatalError, Task, TaskList

Automatic pdb calling has been turned OFF


In [2]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()


Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=74939CnimT4Q-Xy2C7qAirIzL9OGcsRbkXBCYo8yZAw&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!
My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot


In [ ]:
task_id = "3bc6e65c-24f1-11ee-abf4-63e0d97254cd"

In [ ]:
tasks_list = connect_man.get_tasks()
tasks_list_df = tasks_list.to_dataframe()


In [ ]:
tasks_list_df

In [3]:
# [Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'),
#  Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'),
#  Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'),
#  Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'),
#  Bookmark(bookmark_id='1c725da6-203a-11ee-80bc-a3018385fcef', name='Apogee - ProgrammaticDisplayFunctionTesting', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/C/Users/pho/repos/Spike3DWorkEnv/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='4152f10a-a17b-11ed-a2a4-8383522b48d9', name='Apogee R VMs', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/R/FastSwap/Virtual Machines/'),
#  Bookmark(bookmark_id='78ca952e-2586-11ec-9e35-3df4ed83d858', name='Cubix Win10 PhoPy3DPositionAnalysis2021', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/'),
#  Bookmark(bookmark_id='e5c8531c-3b77-11ec-adf2-0748e6f5ffe3', name='Cubix Win10 - PhoPy3DPositionAnalysis2021 - data', endpoint_id='6d0251c4-2585-11ec-9e35-3df4ed83d858', path='/~/repos/PhoPy3DPositionAnalysis2021/data/'),
#  Bookmark(bookmark_id='3844ee50-9826-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - Cloud', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/'),
#  Bookmark(bookmark_id='3995215e-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/'),
#  Bookmark(bookmark_id='561e4ad0-b76a-11ed-a982-5f0e34a3cc4f', name='Diba Lab Workstation 2022 - Cloud - MED-DibaLabDropbox/Data/', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/home/halechr/cloud/Dropbox_Diba_Shared/MED-DibaLabDropbox/Data/'),
#  Bookmark(bookmark_id='ba158112-dfd2-11ed-9b9b-c9bb788c490e', name='Diba Lab Workstation 2022 - GDrive Diba Shared', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/cloud/GDrive_Diba_Shared/Data/'),
#  Bookmark(bookmark_id='55246d18-9ce3-11ed-b577-33287ee02ec7', name='Diba Lab Workstation 2022 - greatlakes homedir', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/greatlakes_halechr/'),
#  Bookmark(bookmark_id='f7bd6c36-9825-11ed-a849-256017f36728', name='Diba Lab Workstation 2022 - MAX', endpoint_id='f418ea94-07aa-11ed-8d83-a54cf61939f8', path='/media/MAX/Data/'),
#  Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/'),
#  Bookmark(bookmark_id='8ce139f8-9d46-11ed-a2a2-8383522b48d9', name='KDIBA Lab Turbo', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/'),
#  Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/'),
#  Bookmark(bookmark_id='248a9a30-164e-11ee-bfba-195c41bc0be4', name='LNX00052_Fedora - MAX', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/media/MAX/'),
#  Bookmark(bookmark_id='af68b1ea-20c7-11ee-818f-195c41bc0be4', name='LNX00052_Fedora - ProgrammaticDisplayFunctionTesting', endpoint_id='af3fcfce-f664-11ed-9a7d-83ef71fbf0ae', path='/home/halechr/repo/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='034e1614-3c19-11ec-90c8-7fa38c35632a', name='NYU Langone Health DTN1 - Buzsakilab Public Datasets', endpoint_id='188a6110-96db-11eb-b7a9-f57b2d55370d', path='/'),
#  Bookmark(bookmark_id='0bc636fc-3997-11ec-9f1c-db52857db67d', name='Pho Personal Laptop - Hiro_Datasets', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/iNeo/Data/Rotation_3_Kamran Diba Lab/DataProcessingProject/'),
#  Bookmark(bookmark_id='1805df24-e618-11ed-9a63-83ef71fbf0ae', name='Pho Personal Laptop - PegasusR6 - Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/Volumes/Pegasus R6/Data/'),
#  Bookmark(bookmark_id='bc4aa210-3b77-11ec-adf2-0748e6f5ffe3', name='Pho Personal Laptop - PhoMatlabDataScripting Exported Data', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/'),
#  Bookmark(bookmark_id='8f77467e-2558-11ec-a0a7-6b21ca6daf73', name='Pho Personal Laptop - PhoPy3DPositionAnalysis', endpoint_id='20c84240-1eb1-11eb-81b7-0e2f230cc907', path='/~/repo/Python Projects/PhoPy3DPositionAnalysis/'),
#  Bookmark(bookmark_id='99ce1e08-bdf1-11ed-8cec-f9fa098153fc', name='UMich Diba Turbo - Pho Directory', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Pho/'),
#  Bookmark(bookmark_id='b1668f08-2036-11ee-80bc-a3018385fcef', name='umich#greatlakes - halechr homedir', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/'),
#  Bookmark(bookmark_id='b0569b9c-2558-11ec-a0a7-6b21ca6daf73', name='umich#greatlakes - halechr home folder', endpoint_id='e0370902-9f48-11e9-821b-02b7a92d8e58', path='/~/'),
#  Bookmark(bookmark_id='f479daf4-2039-11ee-80bc-a3018385fcef', name='umich#greatlakes - ProgrammaticDisplayFunctionTesting', endpoint_id='454f457e-a41b-4807-8775-d132f15a228f', path='/home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/'),
#  Bookmark(bookmark_id='bef81650-96c5-11ed-9b93-19370d280681', name='umms-dibalab DataDen', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/')]

bookmark_list = connect_man.get_bookmarks()
bookmark_list

BookmarkList(DATA_TYPE='bookmark_list', DATA=[Bookmark(bookmark_id='9a8eccc8-2558-11ec-a0a7-6b21ca6daf73', name='aleinbook-linux - PhoPy3DPositionAnalysis', endpoint_id='c3a90494-2555-11ec-a47d-a50ad076c282', path='/~/repo/PhoPy3DPositionAnalysis/'), Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/'), Bookmark(bookmark_id='caf6bf20-9c14-11ed-a29d-8383522b48d9', name='Apogee greatlakes_halechr Folder', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/Data/greatlakes_halechr/'), Bookmark(bookmark_id='e1ef1556-4985-11ed-89d5-ede5bae4f491', name='Apogee Personal Dropbox Active', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/E/Dropbox (Personal)/Active/'), Bookmark(bookmark_id='63655524-164e-11ee-bfba-195c41bc0be4', name='Apogee - PersonalFastSwapShare', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/W/FastSwap/PersonalFastSwapShare/'), Bookmark(bookmar

In [4]:
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
lab_Turbo_data_folder_bookmark

Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

In [5]:
lab_DataDen_data_folder_bookmark = Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')
lab_DataDen_data_folder_bookmark

Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')

In [ ]:
# target_bookmark: Bookmark = lab_DataDen_data_folder_bookmark
target_bookmark: Bookmark = lab_Turbo_data_folder_bookmark
target_bookmark

In [ ]:
file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date=None, end_date=None)
file_list.to_dataframe()

In [ ]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=False)
recent_file_list.to_dataframe() #.columns

In [ ]:
recent_file_list = connect_man.list_files(endpoint=target_bookmark.endpoint_id, path=target_bookmark.path, start_date="2023-08-07", end_date=None, should_list_recursively=True)
recent_file_list # 1m 44.7 sec

In [ ]:
recent_file_list_df = recent_file_list.to_dataframe()
recent_file_list_df

In [ ]:
recent_file_list.path

# Loading filelists:

In [10]:
from phoglobushelpers.path_helpers import convert_filelist_to_new_parent, read_lines_from_file, find_matching_parent_path, known_global_data_root_parent_paths


filelist_source_path = Path('../filelists/fileList_Greatlakes_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)

filelist_dest_path = '../filelists/dest_fileList_Apogee_2023-08-07.txt'
dest_lines = read_lines_from_file(filelist_dest_path)


filelist_source_path = Path('../filelists/fileList_Greatlakes_HDF5_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)


['/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/global_computation_results.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/loadedSessPickle.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/global_computation_results.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/global_computation_results.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-08_21-16-25/loadedSessPickle.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/global_computation_results.pkl',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/20

In [ ]:
filelist_source_path = Path('../filelists/fileList_Greatlakes_pkls_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)
src_lines

In [14]:
filelist_source_path = Path('../filelists/fileList_Greatlakes_HDF5_2023-08-07.txt').resolve()
src_lines = read_lines_from_file(filelist_source_path)
src_lines

['/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/pipeline_results.h5',
 '/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/p

In [15]:
src_lines = [a_line.replace('/nfs/turbo/umms-kdiba/Data/', '/umms-kdiba/Data/', 1) for a_line in src_lines]
src_lines

dest_lines = [a_line.replace('/umms-kdiba/Data/', '/~/W/Data/', 1) for a_line in src_lines]
dest_lines

['/~/W/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/pin01/one/11-02_17-46-44/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/pin01/one/11-02_19-28-0/output/pipeline_results.h5',
 '/~/W/Data/KDIBA/pin01

In [ ]:
from pathlib import Path


curr_filelist = [Path(a_line) for a_line in src_lines]

desired_global_data_root_parent_path = Path('/~/W/Data/')
prev_global_data_root_parent_path = find_matching_parent_path(known_global_data_root_parent_paths, curr_filelist[0]) # TODO: assumes all have the same root, which is a valid assumption so far. ## prev_global_data_root_parent_path should contain the matching path from the list.
assert prev_global_data_root_parent_path is not None, f"No matching root parent path could be found!!"
converted_dest_paths = convert_filelist_to_new_parent(curr_filelist, original_parent_path=prev_global_data_root_parent_path, dest_parent_path=desired_global_data_root_parent_path)
dest_lines = [str(a_path) for a_path in converted_dest_paths]
dest_lines
converted_dest_paths[0]

In [ ]:
from phoglobushelpers.path_helpers import convert_filelist_to_new_parent

# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

In [16]:
apogee_data_folder_bookmark = Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/')
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

# endpoint_relative_src_lines = [a_line.replace('/umms-kdiba/Data/', '/', 1) for a_line in src_lines]
# endpoint_relative_dest_lines = [a_line.replace('/~/W/Data/', '/', 1) for a_line in dest_lines]

endpoint_relative_src_lines = [a_line for a_line in src_lines]
endpoint_relative_dest_lines = [a_line for a_line in dest_lines]
_transfer_task = connect_man.batch_transfer_files(source_endpoint=lab_Turbo_data_folder_bookmark.endpoint_id, destination_endpoint=apogee_data_folder_bookmark.endpoint_id, filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines, max_single_file_wait_time_seconds=60.0)
_transfer_task

Transferring /umms-kdiba/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5...
Transferring /umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5...
Transferring /umms-kdiba/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5...
Transferring /umms-kdiba/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5...
Transferring /umms-kdiba/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5...
Transferring /umms-kdiba/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5 to /~/W/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipel

In [ ]:
unique_rows_df = spikes_df.spikes.extract_unique_neuron_identities()